<a href="https://colab.research.google.com/github/santiagonajera/CAINCO-13-Mayo2025/blob/main/PlanificacionSemanal.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np

# URLs de entrada
url_ventas = "https://github.com/santiagonajera/CAINCO-13-Mayo2025/raw/refs/heads/main/ventas_items_cainco.xlsx "
url_leadtime = "https://github.com/santiagonajera/CAINCO-13-Mayo2025/raw/refs/heads/main/Cainco-Costos-LeadTime.xlsx "

# -------------------------------
# 1. Clasificación ABC (Basada en Valor Monetario)
# -------------------------------

# Cargar ventas anuales (hoja '2026')
ventas_df = pd.read_excel(url_leadtime, sheet_name="2026")
ventas_df.columns = ["MES"] + [f"Item_{i+1}" for i in range(len(ventas_df.columns) - 1)]
ventas_df = ventas_df.drop(columns=["MES"])

# Ventas anuales por producto
ventas_anuales = ventas_df.sum().reset_index()
ventas_anuales.columns = ["Producto", "Ventas_Anuales"]

# Cargar precios (hoja 'Precios-Costos')
precios_df = pd.read_excel(url_leadtime, sheet_name="Precios-Costos")
precios_df["Producto"] = precios_df["Item"]

# Combinar ventas y precios
datos_combinados = pd.merge(ventas_anuales, precios_df, on="Producto")
datos_combinados["Valor_Monetario_Total"] = datos_combinados["Ventas_Anuales"] * datos_combinados["Precio_Venta_USD"]
datos_combinados = datos_combinados.sort_values(by="Valor_Monetario_Total", ascending=False).reset_index(drop=True)

# Calcular porcentaje acumulado
total_valor = datos_combinados["Valor_Monetario_Total"].sum()
datos_combinados["Porcentaje_Acumulado"] = datos_combinados["Valor_Monetario_Total"].cumsum() / total_valor

# Clasificación ABC
def asignar_abc(porcentaje):
    if porcentaje <= 0.6:
        return "A"
    elif porcentaje <= 0.8:
        return "B"
    else:
        return "C"

datos_combinados["Clasificacion_ABC"] = datos_combinados["Porcentaje_Acumulado"].apply(asignar_abc)


# -------------------------------
# 2. Clasificación XYZ (Basada en Coeficiente de Variación de Ventas)
# -------------------------------

# Usar el mismo ventas_df para calcular CV
media = ventas_df.mean()
desviacion = ventas_df.std()
coef_variacion = (desviacion / media) * 100  # En porcentaje

df_cv = pd.DataFrame({
    'Producto': coef_variacion.index,
    'CoeficienteVariacion': coef_variacion.values
})

# Calcular percentiles para clasificar XYZ
percentil_33 = np.percentile(df_cv['CoeficienteVariacion'], 33)
percentil_67 = np.percentile(df_cv['CoeficienteVariacion'], 67)

# Clasificación XYZ
def asignar_xyz(cv):
    if cv <= percentil_33:
        return "X"
    elif cv <= percentil_67:
        return "Y"
    else:
        return "Z"

df_cv["Clasificacion_XYZ"] = df_cv["CoeficienteVariacion"].apply(asignar_xyz)


# -------------------------------
# 3. Cálculo de días óptimos de inventario
# -------------------------------

# Leer datos de ventas por fecha
df_ventas = pd.read_excel(url_ventas)
df_ventas['Fecha'] = pd.to_datetime(df_ventas['Fecha'])

# Leer Lead Time
df_leadtime = pd.read_excel(url_leadtime, sheet_name='LeadTime')

# Estadísticas de ventas
estadisticas_ventas = {}
for item in df_ventas.columns[1:]:
    valores = df_ventas[item]
    promedio = np.mean(valores)
    std = np.std(valores)
    estadisticas_ventas[item] = {
        'Promedio_Ventas': promedio,
        'Desviación_Estándar_Ventas': std
    }

# Estadísticas de LeadTime
estadisticas_leadtime = {}
for item in df_leadtime['Item']:
    item_data = df_leadtime[df_leadtime['Item'] == item]
    lead_times = item_data[['LeadTime_1', 'LeadTime_2']].values.flatten()
    promedio = np.mean(lead_times) / 7  # Semanas
    std = np.std(lead_times)
    estadisticas_leadtime[item] = {
        'Promedio_LeadTime': promedio,
        'Desviación_Estándar_LeadTime': std
    }

# Parámetros
Z_alpha = 1.645  # Nivel de confianza del 95%
R = 0.5 / 7  # Reorder Point en semanas

# Calcular stocks y días de inventario
stocks_seguridad_dias = {}
techo_inventario_dias = {}
stocks_seguridad_optimo_dias = {}

for item in estadisticas_ventas.keys():
    mu_d = estadisticas_ventas[item]['Promedio_Ventas']
    sigma_d = estadisticas_ventas[item]['Desviación_Estándar_Ventas']
    mu_L = estadisticas_leadtime.get(item, {}).get('Promedio_LeadTime', 0)
    sigma_L = estadisticas_leadtime.get(item, {}).get('Desviación_Estándar_LeadTime', 0)

    ss = Z_alpha * np.sqrt((mu_L + R) * sigma_d**2 + sigma_L**2 * mu_d**2)
    ss_dias = (ss / mu_d) * 7
    techo_inventario = ss_dias + (mu_L * 7)
    ss_optimo_dias = ss_dias + ((techo_inventario - ss_dias) / 2)

    stocks_seguridad_dias[item] = ss_dias
    techo_inventario_dias[item] = techo_inventario
    stocks_seguridad_optimo_dias[item] = ss_optimo_dias


# Convertir a DataFrame
df_inventario = pd.DataFrame({
    'Stock_Seguridad_Días': stocks_seguridad_dias,
    'Techo_Inventario_Días': techo_inventario_dias,
    'Stock_Seguridad_Optimo_Días': stocks_seguridad_optimo_dias
}).T.reset_index().melt(id_vars='index').pivot(index='index', columns='variable', value='value').reset_index()
df_inventario.columns = ['Producto', 'Stock_Seguridad_Días', 'Stock_Seguridad_Optimo_Días', 'Techo_Inventario_Días']


# -------------------------------
# 4. Unir todo en una tabla final
# -------------------------------

# Unir ABC y XYZ
resultado_final = pd.merge(
    datos_combinados[["Producto", "Clasificacion_ABC"]],
    df_cv[["Producto", "Clasificacion_XYZ"]],
    on="Producto"
)

# Agregar días de inventario
resultado_final = pd.merge(resultado_final, df_inventario, on="Producto")
resultado_final["Clasificacion_ABCXYZ"] = resultado_final["Clasificacion_ABC"] + resultado_final["Clasificacion_XYZ"]

# Mostrar resultados
print("\n✅ Resultado Final:")
print(resultado_final.round(2))

# Opcional: Exportar a Excel
# resultado_final.to_excel("Clasificacion_ABCXYZ_con_Inventario.xlsx", index=False)

TypeError: DataFrame.pivot() got an unexpected keyword argument 'value'

In [2]:
import pandas as pd
import numpy as np

# URLs de los archivos
url_ventas = "https://github.com/santiagonajera/CAINCO-13-Mayo2025/raw/refs/heads/main/ventas_items_cainco.xlsx "
url_leadtime = "https://github.com/santiagonajera/CAINCO-13-Mayo2025/raw/refs/heads/main/Cainco-Costos-LeadTime.xlsx "

# -------------------------------
# 1. Clasificación ABC (Basada en Valor Monetario)
# -------------------------------

# Cargar datos de ventas anuales (hoja '2026')
ventas_df = pd.read_excel(url_leadtime, sheet_name="2026")
ventas_df.columns = ["MES"] + [f"Item_{i+1}" for i in range(len(ventas_df.columns) - 1)]
ventas_df = ventas_df.drop(columns=["MES"])

# Calcular ventas anuales por producto
ventas_anuales = ventas_df.sum().reset_index()
ventas_anuales.columns = ["Producto", "Ventas_Anuales"]

# Cargar precios (hoja 'Precios-Costos')
precios_df = pd.read_excel(url_leadtime, sheet_name="Precios-Costos")
precios_df["Producto"] = precios_df["Item"]

# Combinar ventas y precios
datos_combinados = pd.merge(ventas_anuales, precios_df, on="Producto")
datos_combinados["Valor_Monetario_Total"] = datos_combinados["Ventas_Anuales"] * datos_combinados["Precio_Venta_USD"]
datos_combinados = datos_combinados.sort_values(by="Valor_Monetario_Total", ascending=False).reset_index(drop=True)

# Calcular porcentaje acumulado
total_valor = datos_combinados["Valor_Monetario_Total"].sum()
datos_combinados["Porcentaje_Acumulado"] = datos_combinados["Valor_Monetario_Total"].cumsum() / total_valor

# Clasificación ABC
def asignar_abc(porcentaje):
    if porcentaje <= 0.6:
        return "A"
    elif porcentaje <= 0.8:
        return "B"
    else:
        return "C"

datos_combinados["Clasificacion_ABC"] = datos_combinados["Porcentaje_Acumulado"].apply(asignar_abc)


# -------------------------------
# 2. Clasificación XYZ (Basada en Coeficiente de Variación de Ventas)
# -------------------------------

# Usar ventas_df para calcular coeficiente de variación
media = ventas_df.mean()
desviacion = ventas_df.std()
coef_variacion = (desviacion / media) * 100  # En porcentaje

df_cv = pd.DataFrame({
    'Producto': coef_variacion.index,
    'CoeficienteVariacion': coef_variacion.values
})

# Calcular percentiles para clasificar XYZ
percentil_33 = np.percentile(df_cv['CoeficienteVariacion'], 33)
percentil_67 = np.percentile(df_cv['CoeficienteVariacion'], 67)

# Clasificación XYZ
def asignar_xyz(cv):
    if cv <= percentil_33:
        return "X"
    elif cv <= percentil_67:
        return "Y"
    else:
        return "Z"

df_cv["Clasificacion_XYZ"] = df_cv["CoeficienteVariacion"].apply(asignar_xyz)


# -------------------------------
# 3. Cálculo de días óptimos de inventario
# -------------------------------

# Leer datos de ventas por fecha
df_ventas = pd.read_excel(url_ventas)
df_ventas['Fecha'] = pd.to_datetime(df_ventas['Fecha'])

# Leer Lead Time
df_leadtime = pd.read_excel(url_leadtime, sheet_name='LeadTime')

# Estadísticas de ventas
estadisticas_ventas = {}
for item in df_ventas.columns[1:]:
    valores = df_ventas[item]
    promedio = np.mean(valores)
    std = np.std(valores)
    estadisticas_ventas[item] = {
        'Promedio_Ventas': promedio,
        'Desviación_Estándar_Ventas': std
    }

# Estadísticas de LeadTime
estadisticas_leadtime = {}
for item in df_leadtime['Item']:
    item_data = df_leadtime[df_leadtime['Item'] == item]
    lead_times = item_data[['LeadTime_1', 'LeadTime_2']].values.flatten()
    promedio = np.mean(lead_times) / 7  # Semanas
    std = np.std(lead_times)
    estadisticas_leadtime[item] = {
        'Promedio_LeadTime': promedio,
        'Desviación_Estándar_LeadTime': std
    }

# Parámetros
Z_alpha = 1.645  # Nivel de confianza del 95%
R = 0.5 / 7  # Reorder Point en semanas

# Calcular stocks y días de inventario
stocks_seguridad_dias = {}
techo_inventario_dias = {}
stocks_seguridad_optimo_dias = {}

for item in estadisticas_ventas.keys():
    mu_d = estadisticas_ventas[item]['Promedio_Ventas']
    sigma_d = estadisticas_ventas[item]['Desviación_Estándar_Ventas']
    mu_L = estadisticas_leadtime.get(item, {}).get('Promedio_LeadTime', 0)
    sigma_L = estadisticas_leadtime.get(item, {}).get('Desviación_Estándar_LeadTime', 0)

    ss = Z_alpha * np.sqrt((mu_L + R) * sigma_d**2 + sigma_L**2 * mu_d**2)
    ss_dias = (ss / mu_d) * 7
    techo_inventario = ss_dias + (mu_L * 7)
    ss_optimo_dias = ss_dias + ((techo_inventario - ss_dias) / 2)

    stocks_seguridad_dias[item] = ss_dias
    techo_inventario_dias[item] = techo_inventario
    stocks_seguridad_optimo_dias[item] = ss_optimo_dias


# Convertir a DataFrame
inventario_list = []
for item in stocks_seguridad_dias:
    inventario_list.append({
        'Producto': item,
        'Stock_Seguridad_Días': stocks_seguridad_dias[item],
        'Techo_Inventario_Días': techo_inventario_dias[item],
        'Stock_Seguridad_Optimo_Días': stocks_seguridad_optimo_dias[item]
    })

df_inventario = pd.DataFrame(inventario_list)


# -------------------------------
# 4. Unir todo en una tabla final
# -------------------------------

# Unir ABC y XYZ
resultado_final = pd.merge(
    datos_combinados[["Producto", "Clasificacion_ABC"]],
    df_cv[["Producto", "Clasificacion_XYZ"]],
    on="Producto",
    how="inner"
)

# Agregar días de inventario
resultado_final = pd.merge(resultado_final, df_inventario, on="Producto", how="inner")
resultado_final["Clasificacion_ABCXYZ"] = resultado_final["Clasificacion_ABC"] + resultado_final["Clasificacion_XYZ"]

# Mostrar resultados
print("\n✅ Resultado Final:")
print(resultado_final.round(2))

# Opcional: Exportar a Excel
# resultado_final.to_excel("Clasificacion_ABCXYZ_con_Inventario.xlsx", index=False)


✅ Resultado Final:
   Producto Clasificacion_ABC Clasificacion_XYZ  Stock_Seguridad_Días  \
0    Item_8                 A                 Y                 23.12   
1   Item_20                 A                 X                  1.20   
2    Item_1                 A                 Y                 40.36   
3   Item_15                 A                 Z                 17.51   
4   Item_21                 A                 Z                 11.77   
5   Item_10                 A                 Z                 34.73   
6   Item_12                 A                 Y                 23.22   
7    Item_5                 A                 X                 63.37   
8   Item_14                 A                 Y                 34.59   
9   Item_16                 A                 Y                  6.10   
10   Item_6                 B                 Z                 57.84   
11   Item_3                 B                 Y                 34.67   
12   Item_7                 B  

In [3]:
import pandas as pd
import numpy as np
from prophet import Prophet
from scipy.stats.mstats import winsorize

# URLs de los archivos
url_ventas = "https://github.com/santiagonajera/CAINCO-13-Mayo2025/raw/refs/heads/main/ventas_items_cainco.xlsx "
url_leadtime = "https://github.com/santiagonajera/CAINCO-13-Mayo2025/raw/refs/heads/main/Cainco-Costos-LeadTime.xlsx "

# -------------------------------
# 1. Clasificación ABC (Basada en Valor Monetario)
# -------------------------------

ventas_df = pd.read_excel(url_leadtime, sheet_name="2026")
ventas_df.columns = ["MES"] + [f"Item_{i+1}" for i in range(len(ventas_df.columns) - 1)]
ventas_df = ventas_df.drop(columns=["MES"])

ventas_anuales = ventas_df.sum().reset_index()
ventas_anuales.columns = ["Producto", "Ventas_Anuales"]

precios_df = pd.read_excel(url_leadtime, sheet_name="Precios-Costos")
precios_df["Producto"] = precios_df["Item"]

datos_combinados = pd.merge(ventas_anuales, precios_df, on="Producto")
datos_combinados["Valor_Monetario_Total"] = datos_combinados["Ventas_Anuales"] * datos_combinados["Precio_Venta_USD"]
datos_combinados = datos_combinados.sort_values(by="Valor_Monetario_Total", ascending=False).reset_index(drop=True)

total_valor = datos_combinados["Valor_Monetario_Total"].sum()
datos_combinados["Porcentaje_Acumulado"] = datos_combinados["Valor_Monetario_Total"].cumsum() / total_valor

def asignar_abc(porcentaje):
    if porcentaje <= 0.6:
        return "A"
    elif porcentaje <= 0.8:
        return "B"
    else:
        return "C"

datos_combinados["Clasificacion_ABC"] = datos_combinados["Porcentaje_Acumulado"].apply(asignar_abc)

# -------------------------------
# 2. Clasificación XYZ (Coeficiente de Variación)
# -------------------------------

media = ventas_df.mean()
desviacion = ventas_df.std()
coef_variacion = (desviacion / media) * 100

df_cv = pd.DataFrame({
    'Producto': coef_variacion.index,
    'CoeficienteVariacion': coef_variacion.values
})

percentil_33 = np.percentile(df_cv['CoeficienteVariacion'], 33)
percentil_67 = np.percentile(df_cv['CoeficienteVariacion'], 67)

def asignar_xyz(cv):
    if cv <= percentil_33:
        return "X"
    elif cv <= percentil_67:
        return "Y"
    else:
        return "Z"

df_cv["Clasificacion_XYZ"] = df_cv["CoeficienteVariacion"].apply(asignar_xyz)

# -------------------------------
# 3. Días óptimos de inventario
# -------------------------------

df_ventas_forecast = pd.read_excel(url_ventas)
df_ventas_forecast['Fecha'] = pd.to_datetime(df_ventas_forecast['Fecha'])

df_leadtime = pd.read_excel(url_leadtime, sheet_name='LeadTime')

estadisticas_ventas = {}
for item in df_ventas_forecast.columns[1:]:
    valores = df_ventas_forecast[item]
    promedio = np.mean(valores)
    std = np.std(valores)
    estadisticas_ventas[item] = {
        'Promedio_Ventas': promedio,
        'Desviación_Estándar_Ventas': std
    }

estadisticas_leadtime = {}
for item in df_leadtime['Item']:
    item_data = df_leadtime[df_leadtime['Item'] == item]
    lead_times = item_data[['LeadTime_1', 'LeadTime_2']].values.flatten()
    promedio = np.mean(lead_times) / 7
    std = np.std(lead_times)
    estadisticas_leadtime[item] = {
        'Promedio_LeadTime': promedio,
        'Desviación_Estándar_LeadTime': std
    }

Z_alpha = 1.645
R = 0.5 / 7

stocks_seguridad_optimo_dias = {}

for item in estadisticas_ventas.keys():
    mu_d = estadisticas_ventas[item]['Promedio_Ventas']
    sigma_d = estadisticas_ventas[item]['Desviación_Estándar_Ventas']
    mu_L = estadisticas_leadtime.get(item, {}).get('Promedio_LeadTime', 0)
    sigma_L = estadisticas_leadtime.get(item, {}).get('Desviación_Estándar_LeadTime', 0)

    ss = Z_alpha * np.sqrt((mu_L + R) * sigma_d**2 + sigma_L**2 * mu_d**2)
    ss_dias = (ss / mu_d) * 7
    techo_inventario = ss_dias + (mu_L * 7)
    ss_optimo_dias = ss_dias + ((techo_inventario - ss_dias) / 2)

    stocks_seguridad_optimo_dias[item] = ss_optimo_dias

# Convertir a DataFrame
inventario_list = []
for item in stocks_seguridad_optimo_dias:
    inventario_list.append({
        'Producto': item,
        'Inventario_Optimo_Dias': stocks_seguridad_optimo_dias[item]
    })

df_inventario = pd.DataFrame(inventario_list)

# -------------------------------
# 4. Forecast de ventas (1 semana)
# -------------------------------

weeks_to_forecast = 1  # Solo 1 semana

future_predictions = {}

for item in df_ventas_forecast.columns[1:]:
    print(f"\nProcesando {item}...")

    series = df_ventas_forecast[item].copy()
    winsorized_series = winsorize(series, limits=[0.01, 0.01])

    prophet_df = pd.DataFrame({
        'ds': df_ventas_forecast['Fecha'],
        'y': winsorized_series
    })

    model = Prophet(
        yearly_seasonality=True,
        weekly_seasonality=True,
        daily_seasonality=False,
        changepoint_prior_scale=0.05,
        seasonality_prior_scale=10.0
    )
    model.add_country_holidays(country_name='Bolivia')
    model.fit(prophet_df)

    future = model.make_future_dataframe(periods=weeks_to_forecast, freq='W')
    forecast = model.predict(future)

    pred_df = forecast[['ds', 'yhat']].tail(weeks_to_forecast)
    future_predictions[item] = pred_df.iloc[0]['yhat']

# Crear DataFrame con pronóstico
df_forecast_semana = pd.DataFrame({
    'Producto': future_predictions.keys(),
    'Venta_Pronosticada_1semana': future_predictions.values()
})

# -------------------------------
# 5. Unir todo en una tabla final
# -------------------------------

resultado_final = pd.merge(
    datos_combinados[["Producto", "Clasificacion_ABC"]],
    df_cv[["Producto", "Clasificacion_XYZ"]],
    on="Producto",
    how="inner"
)

resultado_final = pd.merge(resultado_final, df_inventario, on="Producto", how="inner")
resultado_final = pd.merge(resultado_final, df_forecast_semana, on="Producto", how="inner")
resultado_final["Clasificacion_ABCXYZ"] = resultado_final["Clasificacion_ABC"] + resultado_final["Clasificacion_XYZ"]

# Mostrar resultados
print("\n✅ Resultado Final:")
print(resultado_final.round(2))

# Opcional: Exportar a Excel
# resultado_final.to_excel("Clasificacion_ABCXYZ_Inventario_Forecast.xlsx", index=False)


Procesando Item_1...


DEBUG:cmdstanpy:input tempfile: /tmp/tmpzl0nxv77/tiu8iup1.json
DEBUG:cmdstanpy:input tempfile: /tmp/tmpzl0nxv77/6tpsbx0o.json
DEBUG:cmdstanpy:idx 0
DEBUG:cmdstanpy:running CmdStan, num_threads: None
DEBUG:cmdstanpy:CmdStan args: ['/usr/local/lib/python3.11/dist-packages/prophet/stan_model/prophet_model.bin', 'random', 'seed=20083', 'data', 'file=/tmp/tmpzl0nxv77/tiu8iup1.json', 'init=/tmp/tmpzl0nxv77/6tpsbx0o.json', 'output', 'file=/tmp/tmpzl0nxv77/prophet_model5h5ygwh7/prophet_model-20250515012857.csv', 'method=optimize', 'algorithm=lbfgs', 'iter=10000']
01:28:57 - cmdstanpy - INFO - Chain [1] start processing
INFO:cmdstanpy:Chain [1] start processing
01:28:57 - cmdstanpy - INFO - Chain [1] done processing
INFO:cmdstanpy:Chain [1] done processing
DEBUG:cmdstanpy:input tempfile: /tmp/tmpzl0nxv77/20ts0iwt.json
DEBUG:cmdstanpy:input tempfile: /tmp/tmpzl0nxv77/um12s41x.json
DEBUG:cmdstanpy:idx 0
DEBUG:cmdstanpy:running CmdStan, num_threads: None
DEBUG:cmdstanpy:CmdStan args: ['/usr/local/


Procesando Item_2...


DEBUG:cmdstanpy:input tempfile: /tmp/tmpzl0nxv77/ain9u3z2.json
DEBUG:cmdstanpy:input tempfile: /tmp/tmpzl0nxv77/y985vmx8.json
DEBUG:cmdstanpy:idx 0
DEBUG:cmdstanpy:running CmdStan, num_threads: None
DEBUG:cmdstanpy:CmdStan args: ['/usr/local/lib/python3.11/dist-packages/prophet/stan_model/prophet_model.bin', 'random', 'seed=13061', 'data', 'file=/tmp/tmpzl0nxv77/ain9u3z2.json', 'init=/tmp/tmpzl0nxv77/y985vmx8.json', 'output', 'file=/tmp/tmpzl0nxv77/prophet_model4zh48og2/prophet_model-20250515012857.csv', 'method=optimize', 'algorithm=lbfgs', 'iter=10000']
01:28:57 - cmdstanpy - INFO - Chain [1] start processing
INFO:cmdstanpy:Chain [1] start processing
01:28:57 - cmdstanpy - INFO - Chain [1] done processing
INFO:cmdstanpy:Chain [1] done processing



Procesando Item_3...


...


Procesando Item_23...


DEBUG:cmdstanpy:input tempfile: /tmp/tmpzl0nxv77/o8ftv364.json
DEBUG:cmdstanpy:input tempfile: /tmp/tmpzl0nxv77/a4oswmo1.json
DEBUG:cmdstanpy:idx 0
DEBUG:cmdstanpy:running CmdStan, num_threads: None
DEBUG:cmdstanpy:CmdStan args: ['/usr/local/lib/python3.11/dist-packages/prophet/stan_model/prophet_model.bin', 'random', 'seed=73535', 'data', 'file=/tmp/tmpzl0nxv77/o8ftv364.json', 'init=/tmp/tmpzl0nxv77/a4oswmo1.json', 'output', 'file=/tmp/tmpzl0nxv77/prophet_modelgwniuk0s/prophet_model-20250515012902.csv', 'method=optimize', 'algorithm=lbfgs', 'iter=10000']
01:29:02 - cmdstanpy - INFO - Chain [1] start processing
INFO:cmdstanpy:Chain [1] start processing
01:29:02 - cmdstanpy - INFO - Chain [1] done processing
INFO:cmdstanpy:Chain [1] done processing



Procesando Item_24...


DEBUG:cmdstanpy:input tempfile: /tmp/tmpzl0nxv77/l97qcfg4.json
DEBUG:cmdstanpy:input tempfile: /tmp/tmpzl0nxv77/3xpfjsip.json
DEBUG:cmdstanpy:idx 0
DEBUG:cmdstanpy:running CmdStan, num_threads: None
DEBUG:cmdstanpy:CmdStan args: ['/usr/local/lib/python3.11/dist-packages/prophet/stan_model/prophet_model.bin', 'random', 'seed=73330', 'data', 'file=/tmp/tmpzl0nxv77/l97qcfg4.json', 'init=/tmp/tmpzl0nxv77/3xpfjsip.json', 'output', 'file=/tmp/tmpzl0nxv77/prophet_modelzgy3rmta/prophet_model-20250515012903.csv', 'method=optimize', 'algorithm=lbfgs', 'iter=10000']
01:29:03 - cmdstanpy - INFO - Chain [1] start processing
INFO:cmdstanpy:Chain [1] start processing
01:29:03 - cmdstanpy - INFO - Chain [1] done processing
INFO:cmdstanpy:Chain [1] done processing



Procesando Item_25...

✅ Resultado Final:
   Producto Clasificacion_ABC Clasificacion_XYZ  Inventario_Optimo_Dias  \
0    Item_8                 A                 Y                   27.12   
1   Item_20                 A                 X                    3.20   
2    Item_1                 A                 Y                   44.61   
3   Item_15                 A                 Z                   20.76   
4   Item_21                 A                 Z                   15.77   
5   Item_10                 A                 Z                   40.23   
6   Item_12                 A                 Y                   29.22   
7    Item_5                 A                 X                   67.62   
8   Item_14                 A                 Y                   39.09   
9   Item_16                 A                 Y                    9.85   
10   Item_6                 B                 Z                   62.84   
11   Item_3                 B                 Y          

In [5]:
# Asumimos saldo inicial constante
saldo_inicial = 100

# Calcular cantidad a comprar
resultado_final['Cantidad_Comprar'] = (
    (resultado_final['Venta_Pronosticada_1semana'] / 7) * resultado_final['Inventario_Optimo_Dias']
) - saldo_inicial + resultado_final['Venta_Pronosticada_1semana']

# Redondear a 2 decimales y evitar valores negativos (mínimo 0)
resultado_final['Cantidad_Comprar'] = resultado_final['Cantidad_Comprar'].apply(lambda x: max(round(x), 0))

# Mostrar resultados finales
print("\n✅ Resultado Final con Cantidad a Comprar:")
print(resultado_final[[
    'Producto', 'Clasificacion_ABC', 'Clasificacion_XYZ',
    'Clasificacion_ABCXYZ', 'Inventario_Optimo_Dias',
    'Venta_Pronosticada_1semana', 'Cantidad_Comprar'
]].round(2))


✅ Resultado Final con Cantidad a Comprar:
   Producto Clasificacion_ABC Clasificacion_XYZ Clasificacion_ABCXYZ  \
0    Item_8                 A                 Y                   AY   
1   Item_20                 A                 X                   AX   
2    Item_1                 A                 Y                   AY   
3   Item_15                 A                 Z                   AZ   
4   Item_21                 A                 Z                   AZ   
5   Item_10                 A                 Z                   AZ   
6   Item_12                 A                 Y                   AY   
7    Item_5                 A                 X                   AX   
8   Item_14                 A                 Y                   AY   
9   Item_16                 A                 Y                   AY   
10   Item_6                 B                 Z                   BZ   
11   Item_3                 B                 Y                   BY   
12   Item_7          